In [ ]:
# there'll be a direcory called videoMetaData/
# there'll be folders for each device?
# then there'll be folders for each kind of processing
#   time indexes, and why not, lets also put in file name and frame index
#   just poses for now
    # in the format time index, pose list object

# also for the cameras lets build them up one use case at a time
# general tracking multiclass classification
#   sitting - bedroom
#   standing
#   lying down
#   showering - bathroom
#   pooping
#   taking a shower
#   brushing teeth
#   putting corizone on
#   making food - kitchen
#   wiping nose - all

# later I'll have a different set of cameras to help with facial analysis
# I'll still have one recording I can test some processing strategies
# at linux computer

In [ ]:
# let's brainstorm version 2 of this process
# all of the cameras are sending to a folder called recent captures
# for now I'll just run the intake scrip manually

# get the list of folder names

# for each folder
# get the device name from the folder name
# get a list of filenames without type suffixes

# for each of names
# generate the desired location of the video
# make sure the file location for the video is created
# move the video there

# these will be parquets with millions of lines
# honestly I don't want to be opening them all the time
# or even really leaving them open

# nahh let's split by day that's honestly better in so many ways than by month
# yeah the smallest partquets would be like 4k rows but that's fine
# now a question lets just leave the files named appropriately in the month folder
# instead of further in the day folders the videos are in
# name for the parquet will be classic DEVICENAME-YYYY-MM-DD.parquet
# the columns will be - timestamp - fileLocation(YYYY-MM/DD/DN_TS_TS.mp4) - frameNumberInFile
# i guess later on it would make sense to put more frame specific metadata like exposure



# a remider of the file structure
# videoProcessing
# videoData
# |___testCam
#     |___YYYY-MM(-)ZZZZ
#         |___testCam-YYYY-MM-DD(-)ZZZZ/
#         |   |___testCam-TS-TS.mp4
#         |   |___...
#         |___...
#         |___testCam-YYYY-MM(-)ZZZZ.parquet
#         |___...
# recentCaptures
# |___testCam-YYYY-MM-DD(-)ZZZZ/
#     |___testCam-TS-TS.mp4
#     |___testCam-TS-TS.parquet
#     |___...


In [ ]:
import os
import shutil
import pandas as pd
#get list of folders in recent captures
basePath = "/home/chowder/Documents/"
rcFolders = os.listdir(basePath + "recentCaptures/")

# for each folder
for folderName in rcFolders:
    deviceName = folderName.split("-")[0]
    tzOffset = folderName[-5:]
    monthFolderName = "-".join(folderName.split("-")[1:2]) + tzOffset
    baseTarget =  basePath + "/".join(['videoData', deviceName, monthFolderName, ""])

    newDayFolder = basePath + "recentCaptures/" + folderName + "/"
    fileIdentifiers = list(set([f.split(".")[0] for f in os.listdir(newDayFolder)]))
    
    #move mp4's
    for fid in fileIdentifiers:
        source = newDayFolder + fid + ".mp4"
        destination = baseTarget + folderName + "/" + fid + ".mp4"
        shutil.move(source, destination)

    # generate a parquet to add
    for i, fid in enumerate(fileIdentifiers):
        source = newDayFolder + fid + ".parquet"
        readDf = pd.read_parquet(source)
        times = readDf.sampleDT.values
        
        dataToAdd = {}
        dataToAdd['sampleDT'] = times
        dataToAdd['frameNum'] = [x for x in range(len(times))]
        fileLocation = "/".join([deviceName, monthFolderName, folderName, (fid+".mp4")])
        dataToAdd['mp4Location'] = [fileLocation] * len(times)
        
        if i == 0:
            dayDf = pd.DataFrame(dataToAdd)
        else:
            dayDf.concat(pd.DataFrame(dataToAdd))

        os.remove(source)

    # write parquet
    # update this to a util function that's a generic version of the fibit one
    destination = baseTarget + folderName + "/" + monthFolderName + ".parquet"
    if os.path.exists(destination):
        existingDf = pd.read_parquet(destination)
        existingDf.concat(dayDf)
        existingDf.to_parquet(destination)
    else:
        dayDf.to_parquet(destination)
    
    os.rmdir(newDayFolder)
